In this programming exercise, you will implement the validation set approach and K-fold cross validation, while recalling that leave-one-out cross validation is K-fold cross validation for K = N , if N is the number of training samples. Implement the study carried out in Example 6.3 from the lecture notes. Note that due to randomization, you might get other results than the ones shown in the example. 

In [3]:
import numpy as np
import pandas as pd
import random

In [20]:
## Generate random input sample
N = 51
x = []
y = []

for i in range(0, N):
    mu, sigma = 0, 1.15 ## mean and standard deviation
    noise = np.random.normal(mu, sigma, 1)
    
    temp_x = random.randint(-4,4)
    temp_y = temp_x**2 + noise
    
    
    x.append(temp_x)
    y.append(temp_y)



[2, 3, 1, 4, 3, -3, 3, 2, -3, 1, -3, -3, 3, 1, 1, 1, 3, -4, -3, -4, -2, 2, 1, -4, -1, 0, 0, -1, -3, -1, 4, 1, 2, -4, -1, -1, 4, -3, -1, -2, 3, -3, -2, -1, -4, -2, 0, 4, 4, 4, -1] [array([4.15812602]), array([8.35564254]), array([1.50404811]), array([15.69506044]), array([9.32723015]), array([9.59436017]), array([10.27650747]), array([5.96925745]), array([5.21067765]), array([0.04800366]), array([11.30750399]), array([8.39982439]), array([10.20500596]), array([1.7580854]), array([1.34174016]), array([-0.33652715]), array([7.82064715]), array([15.41374031]), array([8.57336155]), array([14.10541213]), array([4.39280979]), array([3.35449484]), array([2.75391545]), array([16.98316058]), array([3.38228223]), array([-1.62878915]), array([2.72240877]), array([0.29495396]), array([8.96008348]), array([1.6346621]), array([16.51947021]), array([0.08598866]), array([3.53456281]), array([16.58244586]), array([-0.65091069]), array([0.89462343]), array([17.90694149]), array([11.68519558]), array([0.7